In [1]:
using Pkg
using Knet: Knet, param, param0, @diff, grad,RNN,mat, params, KnetArray, conv4, Data, relu, pool, dropout
using Random: shuffle!
using IterTools: ncycle
using TestImages,Images, ImageView
using PyCall
@pyimport pickle
@pyimport numpy

┌ Warning: `@pyimport foo` is deprecated in favor of `foo = pyimport("foo")`.
│   caller = _pywrap_pyimport(::PyObject) at PyCall.jl:407
└ @ PyCall /home/okolukisa/.julia/packages/PyCall/ttONZ/src/PyCall.jl:407


In [2]:
# open("instructions.txt") do f
#     for line in eachline(f)
# #         line = split(strip(line),",")
# #         label = [line[2]]
# #         sentence = map(x-> lowercase(x),split(strip(line[1])," "))
# #         append!(instructions,(sentence,label))
#         sentence, label = split(strip(lowercase(line)), ",")
#         wordids = w2i.(split(sentence))
#         labelid = t2i.(label)
# #         println(labelid," ",sentence )
#         push!(instructions, (wordids, labelid))
# #         push!(words, split(strip(sentence), " "))
#     end
# #      print(instructions)
# #     print(words)
# end


In [3]:
wdict = Dict()
tdict = Dict()
w2i(x) = get!(wdict, x, 1+length(wdict))
t2i(x) = get!(tdict, x, 1+length(tdict))
UNK = w2i("<unk>")

1

In [4]:
function readdata(file)
    data = Tuple{Array{Int64,1},Int64}[]
    for line in eachline(file)
        sentence, label = split(strip(lowercase(line)), ",")
        wordids = w2i.(split(sentence))
        labelid = t2i(label)
        push!(data, (wordids, labelid))
    end
    return data
end

readdata (generic function with 1 method)

In [5]:
instructions = readdata("instructions.txt")
first(instructions)

([2, 3, 4, 5], 1)

In [6]:
function prepare_sentence(sent, to_ix)
    sent = split(strip(lowercase(sent),' '))
    idxs = [to_ix[w] for w in sent]
    return idxs
end

prepare_sentence (generic function with 1 method)

In [7]:
w2i(x) = get(wdict, x, UNK)     # unk if not found
t2i(x) = tdict[x]       

t2i (generic function with 1 method)

In [8]:
# function prepare_sentence(sent, to_ix)
#     sent = split(strip(lowercase(sent))," ")
#     idxs = [to_ix[w] for w in sent]
#     return torch.tensor(idxs)
# end

In [9]:
EMBEDDING_DIM = 20
HIDDEN_DIM_LSTM = 10
VOCAB_SIZE = length(wdict)
LABEL_SIZE = length(tdict)

# text_model = LSTMClassifier()
# image_model = ConvNetClassifier()
# loss_function = nn.CosineEmbeddingLoss()
# optimizer1 = optim.SGD(text_model.parameters(), lr = 0.001)
# optimizer2 = optim.SGD(image_model.parameters(), lr = 0.001)

7

In [10]:

function mypickle(filename, obj)
    out = open(filename,"w")
    pickle.dump(obj, out)
    close(out)
 end

function myunpickle(filename)
    r = nothing
    @pywith pybuiltin("open")(filename,"rb") as f begin
        r = pickle.load(f)
    end
    return r
end

myunpickle (generic function with 1 method)

In [11]:

#Test

# dataset = myunpickle("dataset/dataset_true.pickle")

# for epoch in 1:101
#     for dtttt in dataset[1:301]
#         (frame1,frame2), sentence = dtttt
#         println(prepare_sentence(sentence, wdict))
#     end
# end

In [12]:
l = myunpickle("dataset/dataset_true.pickle")

wdict2 = Dict()
tdict2 = Dict()
w2i2(x) = get!(wdict2, x, 1+length(wdict2))
t2i2(x) = get!(tdict2, x, 1+length(tdict2))
UNK2 = w2i2("<unk>")

1

In [13]:
# function GetPreProcessedImageArray(img, cropfromtop::Float64, cropfrombottom::Float64, target_height::Int, target_width::Int)
#     img = img ./ 255
#     height, width, channels = size(img)
#     state_image = permutedims(img,(1,2,3))
#     state_image2 = (state_image[:,Int(height*cropfromtop)+1:Int((1-cropfrombottom)*height),:])
#     grayscale = Gray.(state_image2)
# #     reshape(Float64.(imresize(grayscale, (1,target_height, target_width))), (target_width,target_height))
# end

In [14]:
images = []
im_sentences = []
for i in 1:length(l)
    (f1,f2), sent = l[i]
    #both = numpy.hstack((f1,f2))
    # append!(images,split(strip(sent)," "))
    #resf1 = permutedims((f1[:,:,:])/255,(3,1,2))
    #imshow(resf1)
    #resf2 = permutedims((f1[:,:,:])/255,(3,1,2))
    both = hcat(f1,f2)
#     tst = GetPreProcessedImageArray(both, 0.0, 0.0, 210,320)
#     println(size(tst))
#     imshow(tst)
#     break
    
#     grayf1 = Gray.(resf1)
#     grayf2 = Gray.(resf2)
#     donef1 = reshape(Float64.(imresize(grayf1, (1,210, 60))), 210*60)
#     donef2 = reshape(Float64.(imresize(grayf2, (1,210, 60))), 210*60)
#     both = hcat(donef1,donef2)
#     both = hcat(grayf1,grayf2)
#     print(size(grayf1))
  

#     When we change it to (210,320) it will work better I guess, now its just changed to Float64
    b3 = reshape(Float32.(both), (210,320,3,1))
#         println(typeof(b3))
    
    wordids = w2i2.(split(sent))
    #labelid = t2i(label)
    push!(im_sentences, wordids)
    push!(images,b3)
end
# println(images)

In [15]:
typeof(images[1])

Array{Float32,4}

In [16]:
# Define a convolutional layer:
struct Conv; w; b; f; p; end
(c::Conv)(x) = c.f.(pool(conv4(c.w, dropout(x,c.p)) .+ c.b))
Conv(w1::Int,w2::Int,cx::Int,cy::Int,f=relu;pdrop=0) = Conv(param(w1,w2,cx,cy), param0(1,1,cy,1), f, pdrop)

Conv

In [17]:
struct Convp; w; b; f; p; end
(c::Convp)(x) = c.f.(conv4(c.w, dropout(x,c.p)) .+ c.b)
Convp(w1::Int,w2::Int,cx::Int,cy::Int,f=relu;pdrop=0) = Convp(param(w1,w2,cx,cy), param0(1,1,cy,1), f, pdrop)

Convp

In [18]:
# Redefine dense layer (See mlp.ipynb):
struct Dense; w; b; f; p; end
(d::Dense)(x) = d.f.(d.w * mat(dropout(x,d.p)) .+ d.b) # mat reshapes 4-D tensor to 2-D matrix so we can use matmul
Dense(i::Int,o::Int,f=relu;pdrop=0) = Dense(param(o,i), param0(o), f, pdrop)

Dense

In [19]:
struct Densep; w; b; f; p; end
(d::Densep)(x) = d.f.(d.w * mat(dropout(x,d.p)) .+ d.b) # mat reshapes 4-D tensor to 2-D matrix so we can use matmul
Densep(i::Int,o::Int,f=relu;pdrop=0) = Densep(param(o,i), param0(o), f, pdrop)

Densep

In [20]:
# Let's define a chain of layers
struct Chain
    layers
    Chain(layers...) = new(layers)
end
(c::Chain)(x) = (for l in c.layers; x = l(x); end; x)
(c::Chain)(x,y) = nll(c(x),y)
(c::Chain)(d::Data) = mean(c(x,y) for (x,y) in d)

In [21]:
#Lets get a try in

testChain = Chain(
    Conv(5,5,3,32),#103*158
    Conv(5,5,32,32),#49*77
    Conv(4,4,32,64),#23*37
    Convp(3,3,64,64),#21*35*64 = 47040
    Dense(47040,10,pdrop=0.3))
summary.(l.w for l in testChain.layers)


5-element Array{String,1}:
 "5×5×3×32 AutoGrad.Param{Array{Float32,4}}" 
 "5×5×32×32 AutoGrad.Param{Array{Float32,4}}"
 "4×4×32×64 AutoGrad.Param{Array{Float32,4}}"
 "3×3×64×64 AutoGrad.Param{Array{Float32,4}}"
 "10×47040 AutoGrad.Param{Array{Float32,2}}" 

In [22]:
agent_embedding= []
for i in 1:347
 push!(agent_embedding, testChain(images[i]))
end


In [23]:
struct Embed; w; end
Embed(vocabsize::Int,embedsize::Int) = Embed(param(embedsize,vocabsize))
(e::Embed)(x) = e.w[:,x]

In [42]:
EMBEDDING_DIM = 20
HIDDEN_DIM_LSTM = 10
VOCAB_SIZE = length(wdict)
LABEL_SIZE = length(tdict)

BATCHSIZE = 6
SEQLENGTH = 3;


In [43]:
function seqbatch(x,y,B,T)
    N = length(y) ÷ B
    print(N)
    x = permutedims(reshape(x[1:N*B],N,B))
#     print(x)
    y = permutedims(reshape(y[1:N*B],N,B))
    d = []; for i in 0:T:N-T
        push!(d, (x[:,i+1:i+T], y[:,i+1:i+T]))
    end
    return d
end

seqbatch (generic function with 1 method)

In [44]:
allw = vcat((x->x[1]).(instructions)...)
allt = vcat((x->x[2]).(instructions)...)
d = seqbatch(allw, allt, BATCHSIZE, SEQLENGTH);
d

10

3-element Array{Any,1}:
 ([2 3 4; 4 5 2; … ; 4 8 7; 4 13 11], [1 1 1; 2 3 4; … ; 6 6 6; 6 6 7])  
 ([5 2 3; 3 4 5; … ; 4 9 7; 12 4 13], [1 1 1; 5 5 5; … ; 6 6 6; 7 7 7])  
 ([4 5 2; 2 3 4; … ; 4 10 11; 11 12 4], [1 1 1; 5 6 6; … ; 6 6 6; 7 7 2])

In [45]:
(x,y) = first(d)

([2 3 4; 4 5 2; … ; 4 8 7; 4 13 11], [1 1 1; 2 3 4; … ; 6 6 6; 6 6 7])

In [46]:
embedlayer = Embed(length(instructions),8)


Embed(P(Array{Float32,2}(8,60)))

In [47]:
# shuffle!(d)
# dtst = d[1:10]
# dtrn = d[11:end]
# length.((dtrn,dtst))

In [48]:
# For running experiments we will use the Adam algorithm which typically converges faster than SGD.
function trainresults(file,maker,savemodel)
    if (print("Train from scratch? "); readline()[1]=='y')
        model = maker()
        results = ((nll(model,dtst), zeroone(model,dtst))
                   for x in takenth(progress(adam(model,ncycle(dtrn,5))),100))
        results = reshape(collect(Float32,flatten(results)),(2,:))
        Knet.save(file,"model",(savemodel ? model : nothing),"results",results)
        Knet.gc() # To save gpu memory
    else
        isfile(file) || download("http://people.csail.mit.edu/deniz/models/tutorial/$file",file)
        model,results = Knet.load(file,"model","results")
    end
    println(minimum(results,dims=2))
    return model,results
end

trainresults (generic function with 1 method)

In [49]:
Tagger2(vocab,embed,hidden,output)=  # biRNN Tagger
    Chain(Embed(vocab,embed),RNN(embed,hidden,rnnType=:relu,bidirectional=true),Dense(2hidden,output));

In [50]:
VOCABSIZE = length(wdict)
EMBEDSIZE = 128
HIDDENSIZE = 128
OUTPUTSIZE = length(tdict)

7

In [51]:
t2maker() = Tagger2(VOCABSIZE,EMBEDSIZE,HIDDENSIZE,OUTPUTSIZE)
(t2,r2) = trainresults("instructions.txt",t2maker,true);

Train from scratch? stdin> y


UndefVarError: UndefVarError: dtrn not defined